In [39]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

In [46]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt" #url with formatting to fill in the number of the particular week
    dfs = []
    for week_num in week_nums: #for each week
        file_url = url.format(week_num) #format the url so it grabs that week
        dfs.append(pd.read_csv(file_url)) #append to our empty list
    return pd.concat(dfs) #combine files for each week


start_date = '12/29/2018'
num_per = 24
data_collection_period = pd.date_range(start = start_date, periods = num_per, freq = 'W-SAT')

week_nums = []
for item in data_collection_period:
    week_nums.append(str(item.date()).replace('-','')[2:])

turnstiles_df = get_data(week_nums) #use our function to get a df with all of our data

In [47]:
csv_filename = f'mta_data_{week_nums[0]}_to_{week_nums[-1]}.csv'
turnstiles_df.to_csv(csv_filename, header = False, index = False)

In [48]:
engine = create_engine("sqlite:///mta.db")

In [ ]:
all_data = pd.read_sql('SELECT * FROM mta_data_2019;', engine)

In [50]:
turnstile_count_by_station = pd.read_sql('SELECT COUNT(DISTINCT SCP), STATION, LINENAME \
                                        FROM mta_data_2019 \
                                        GROUP BY STATION, LINENAME \
                                        ORDER BY COUNT(DISTINCT SCP) DESC;', engine)
turnstile_count_by_station.head()

,COUNT(DISTINCT SCP),STATION,LINENAME
0,60,FULTON ST,2345ACJZ
1,50,34 ST-PENN STA,ACE
2,47,PATH NEW WTC,1
3,41,34 ST-HERALD SQ,BDFMNQRW
4,39,CORTLANDT ST,RNW


In [51]:
station_contains_brooklyn = pd.read_sql("SELECT STATION, LINENAME \
                                        FROM mta_data_2019 \
                                        WHERE STATION LIKE '%BROOKLYN%';", engine)
station_contains_brooklyn.head()

,STATION,LINENAME
0,BROOKLYN BRIDGE,456JZ
1,BROOKLYN BRIDGE,456JZ
2,BROOKLYN BRIDGE,456JZ
3,BROOKLYN BRIDGE,456JZ
4,BROOKLYN BRIDGE,456JZ


In [52]:
grand_central_4567S_entries = pd.read_sql("SELECT * \
                                        FROM mta_data_2019 \
                                        WHERE STATION LIKE '%GRD CNTRL%' AND LINENAME = '4567S';", engine)

grand_central_4567S_entries.head()

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,12/22/2018,00:00:00,REGULAR,1754627,4853193
1,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,12/22/2018,04:00:00,REGULAR,1754631,4853198
2,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,12/22/2018,08:00:00,REGULAR,1754645,4853299
3,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,12/22/2018,12:00:00,REGULAR,1754762,4853695
4,R236,R045,00-00-00,GRD CNTRL-42 ST,4567S,IRT,12/22/2018,16:00:00,REGULAR,1754981,4854292
